In [ ]:
!which python; python -V;
from astropy import units as u

from perylune.orbit_tools import *
from perylune.interplanetary import *
from perylune.mpc import *

import time

In [ ]:
# First step is to load the orbit data. parse_txt() will load the orbital elements from text file in MPC format.
# This will be returned as an array. Each entry in that array has a list of 8 parameters or so.
elements = parse_txt("../data/mpc/mpcorb_extended.dat", limit=0, skip="------------------")

print("Number of orbital elements parsed: %d" % len(elements))

In [ ]:
# Now process the data. Find min and max a and e, also put the a and e values in their separate arrays (a_tbl, e_tbl)

max_e = 0
min_e = 9999999
max_a = 0
min_a = 9999999

a_tbl = []
e_tbl = []
cnt = 0
for e in elements:
    a = e[8].value
    ecc = e[6].value
    if a < 6:
        cnt = cnt + 1
        a_tbl.append(a)
        e_tbl.append(e)
    if max_a < a:
        max_a = a
    if min_a > a:
        min_a = a
    if max_e < ecc:
        max_e = ecc
    if min_e > ecc:
        min_e = ecc

print ("Processed %d asteroids, a=%f..%f [AU], eccentricity = %f..%f" % (cnt, min_a, max_a, min_e, max_e))

In [ ]:
# This is the major processing part (that sadly takes a lot of time to complete). It's possible to limit
# the number of asteroids processed by setting max to some value e.g. 20000. However, this will process
# the first 20000 lines from the input file and they're in random (or historical detection) order.
#
# The result is stored in dv_tbl array.
max = 2000000
t0 = time.time()
i = 0

dv_tbl = []
print("Starting calculation, time %s" % time.time())
for e in elements:
    dv_burn1, dv_burn2, dv, tof = calc_min_delta_v(1.0*u.au, e)
    dv_tbl.append([e[0], dv, tof, dv_burn1, dv_burn2])
    i = i + 1
    if (i % 10000 == 0):
        print("Calculated %d orbits, time %s" % (i, time.time()))
    if i >= max:
        break
t1 = time.time()

print("Calculated for %d orbits, calculations took %s s." % (i, (t1-t0)))

In [ ]:
# Now sort the table by the second (index 1) value. This is the total delta-v
t2 = time.time()
dv_tbl2 = sorted(dv_tbl, key=lambda e: e[1].value)
t3 = time.time()

print("dv_tbl (%d elements) is now sorted, sorting took %s s." % (len(dv_tbl), t3-t2))

In [ ]:
# Now print the top 10 asteroids (with smallest delta-v)
top_x = 10
print(len(dv_tbl2))
print(" TOP %d ASTEROIDS THAT ARE EASIEST TO REACH" % top_x)
for i in range(top_x):
    x = dv_tbl2[i]
    print("%d. %s:  fly-by delta-v=%s randezvous delta-v=%s" % (i+1, x[0], x[3], x[1] ))

In [ ]:
# And write all the data to output file.
f = open("../data/mpc/asteroids.txt", "w+")
f.write("#name, total delta-v[m/s], time of flight[days], Hohmann burn1 [m/s], Hohmann burn2 [m/s]\n")
for x in dv_tbl2:
    f.write("%s, %4.2f, %4.1f, %4.2f, %4.2f\n" % (x[0], x[1].value, x[2].value, x[3].value, x[4].value))
f.close()